**Introduction**

This is a Google Colab Notebook which trains a X3D model (using pytorchvideo) on a dataset of videos. The dataset's directory should have 3 sub-directories, one for "train", "val", and "test". In each of these directories should be more directories, named with the dataset's classes. Then, within each of these directories should be videos, whose label corresponds to the directory they are placed in (e.g. dataset->train->archery->archery_1.mph).

The "Set-up" section is used for downloading necessary packages and intiliazing variables. Take a look at the variables (such as file paths), and change them accordingly.

The "Class Creation for Dataloaders" section creates the classes/objects necessary for creating and using dataloaders later in the process. Within this section is a commented code block which would change the transform function to one of a resnet. Uncommenting this code block, and changing the model to "make_resnet" in "Model Creation" would use a resnet50 instead.

The "Model Creation" section creates the object that is used as the Lightning Module for the model.

The "Training" section trains the model using the Trainer from Pytorch Lightning.

The "Testing" section tests the model using Trainer.test and a script that manually tests every file in the testing dataset.

# Set-up


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
!pip install pytorchvideo
!pip install pytorch_lightning

In [ ]:
import os
import pytorch_lightning
import pytorchvideo.data
import torch.utils.data
import torchvision
import pytorchvideo
import pytorchvideo.transforms
from torch.nn.functional import softmax
from typing import Dict
import json
import urllib
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)
import sklearn
#from sklearn.metrics import accuracy_score

In [ ]:
input_dir = '/content/drive/MyDrive/Research/Muscle Video/Datasets/split_workout_videos_v1'
checkpoint_path = '/content/drive/MyDrive/Research/Muscle Video/Checkpoints/Multilabel/x3d_s/not pretrained'
model_name = 'x3d_s'

num_classes=16
num_labels=11
batch_size = 8
num_workers = 10
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
model_transform_params  = {
    "x3d_xs": {
        "side_size": 182,
        "crop_size": 182,
        "num_frames": 4,
        "sampling_rate": 12,
    },
    "x3d_s": {
        "side_size": 182,
        "crop_size": 182,
        "num_frames": 13,
        "sampling_rate": 6,
    },
    "x3d_m": {
        "side_size": 256,
        "crop_size": 256,
        "num_frames": 16,
        "sampling_rate": 5,
    }
}
transform_params = model_transform_params[model_name]
frames_per_second = 30
slowfast_alpha = 4
clip_duration = (transform_params["num_frames"] * transform_params["sampling_rate"])/frames_per_second
device=('cuda' if torch.cuda.is_available() else 'cpu')
pretrained=True
learning_rate=0.0001
dropout_rate = 0.7
gamma = 2

In [ ]:
import multiprocessing

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

12

In [ ]:
pred_to_class = {
    0: "triceps",
    1: "lats",
    2: "biceps",
    3: "quads",
    4: "glutes",
    5: "shoulders",
    6: "abs",
    7: "obliques",
    8: "chest",
    9: "lower back",
    10: "hamstrings",
}

id_to_exercise = {
    0: "bench press",
    1: "bicep curl",
    2: "chest fly machine",
    3: "deadlift",
    4: "hip thrust",
    5: "lat pulling",
    6: "lateral raise",
    7: "leg extension",
    8: "leg raises",
    9: "push-up",
    10: "russian twist",
    11: "shoulder press",
    12: "squat",
    13: "t bar row",
    14: "tricep Pushdown",
    15: "tricep dips",
}

class_to_label = {
    0: [8, 5, 0],
    1: [2],
    2: [8],
    3: [4, 9, 10],
    4: [4],
    5: [1, 2],
    6: [5],
    7: [3],
    8: [6],
    9: [8, 5, 0],
    10: [6, 7],
    11: [5],
    12: [3, 4, 10],
    13: [1, 2],
    14: [0],
    15: [0],
}

id_to_label = {
    0: [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0],
    1: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
    3: [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0],
    4: [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    5: [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    6: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    7: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    8: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
    9: [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0],
    10: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0],
    11: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    12: [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    13: [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    14: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    15: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
}

In [ ]:
a = torch.tensor([12, 5, 14, 9, 10, 15, 12, 3])
new_label = torch.tensor([id_to_label[i.item()] for i in a])
print(new_label)

tensor([[0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.]])


# Class Creation for Dataloaders

In [ ]:
transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(transform_params['num_frames']),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=transform_params['side_size']),
            CenterCropVideo(
                crop_size=(transform_params['crop_size'], transform_params['crop_size'])
            )
        ]
    ),
)

In [ ]:
class VideosDataModule(pytorch_lightning.LightningDataModule):

    # Dataset configuration
    _DATA_PATH = input_dir
    _CLIP_DURATION = clip_duration  # Duration of sampled clip for each video
    _BATCH_SIZE = batch_size
    _NUM_WORKERS = num_workers  # Number of parallel processes fetching data

    def train_dataloader(self):
        #Create the train partition from the list of video labels and video paths
        train_dataset = pytorchvideo.data.labeled_video_dataset(
            data_path=os.path.join(self._DATA_PATH, 'train'),
            clip_sampler=pytorchvideo.data.make_clip_sampler("random", self._CLIP_DURATION),
            decode_audio=False,
            transform=transform
        )

        return torch.utils.data.DataLoader(
            train_dataset,
            batch_size=self._BATCH_SIZE,
            num_workers=self._NUM_WORKERS
        )

    def val_dataloader(self):
        #Create the validation partition from the list of video labels and video paths
        val_dataset = pytorchvideo.data.labeled_video_dataset(
            data_path=os.path.join(self._DATA_PATH, 'val'),
            clip_sampler=pytorchvideo.data.make_clip_sampler("random", self._CLIP_DURATION),
            decode_audio=False,
            transform=transform
        )

        return torch.utils.data.DataLoader(
            val_dataset,
            batch_size=self._BATCH_SIZE,
            num_workers=self._NUM_WORKERS,
        )

    def test_dataloader(self):
        test_dataset = pytorchvideo.data.labeled_video_dataset(
            data_path=os.path.join(self._DATA_PATH, 'test'),
            clip_sampler=pytorchvideo.data.make_clip_sampler("random", self._CLIP_DURATION),
            decode_audio=False,
            transform=transform
        )

        return torch.utils.data.DataLoader(
            test_dataset,
            batch_size=self._BATCH_SIZE,
            num_workers=self._NUM_WORKERS,
        )

# Model Creation


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchmetrics.classification import MultilabelAccuracy, MultilabelAUROC, MultilabelRecall, MultilabelPrecision, MultilabelF1Score

mcaM2 = MultilabelAccuracy(num_labels=num_labels, average='micro')
mcaS = MultilabelAccuracy(num_labels=num_labels, average='none')
roc_auc = MultilabelAUROC(num_labels=num_labels, average="micro")
pre = MultilabelPrecision(num_labels=num_labels, average='macro', threshold=0.5)
rec = MultilabelRecall(num_labels=num_labels, average='macro', threshold=0.5)
f1score = MultilabelF1Score(num_labels=num_labels, average='macro', threshold=0.5)

def post_act(input):
  return softmax(input, dim=1)

class VideoClassificationLightningModule(pytorch_lightning.LightningModule):

    def __init__(self):
        super().__init__()

        self.model = torch.hub.load("facebookresearch/pytorchvideo", model=model_name, pretrained=False)
        self.model.to(device)
        '''
        for param in self.model.parameters():
          param.requires_grad = False
        '''
        self.model.blocks[5].proj = nn.Linear(in_features=2048, out_features=11, bias=True)
        '''
        for param in self.model.blocks[6].proj.parameters():
          param.requires_grad = True
        '''
        self.model.train()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # The model expects a video tensor of shape (B, C, T, H, W), which is the
        # format provided by the dataset
        y_hat = self.model(batch["video"])

        # Compute cross entropy loss, loss.backwards will be called behind the scenes
        # by PyTorchLightning after being returned from this method.

        new_label = torch.tensor([id_to_label[i.item()] for i in batch["label"]])
        loss = F.cross_entropy(y_hat.cpu(), new_label.cpu())

        # Log the train loss to Tensorboard
        self.log("train_loss", loss.item(), batch_size=batch_size)

        accM2 = mcaM2(y_hat.cpu(), new_label.cpu())
        #accS = mcaS(y_hat.cpu(), new_label.cpu())
        self.log("train_accuracy_micro", accM2.item(), batch_size=batch_size)

        print("train_loss:", loss.item(), "train_accuracy_micro:", accM2.item())
        #print("Exercise:", batch["label"], "Unaveraged Accuracy:", accS)

        return loss

    def validation_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])
        new_label = torch.tensor([id_to_label[i.item()] for i in batch["label"]])
        loss = F.cross_entropy(y_hat.cpu(), new_label.cpu())

        self.log("val_loss", loss.item(), batch_size=batch_size)

        accM2 = mcaM2(y_hat.cpu(), new_label.cpu())
        #accS = mcaS(y_hat.cpu(), new_label.cpu())
        self.log("val_accuracy_micro", accM2.item(), batch_size=batch_size)

        print("val_loss:", loss.item(), "val_accuracy_micro:", accM2.item())
        #print("Exercise:", batch["label"], "Unaveraged Accuracy:", accS)

        return loss

    def configure_optimizers(self):
        """
        Setup the Adam optimizer. Note, that this function also can return a lr scheduler, which is
        usually useful for training video models.
        """
        return torch.optim.Adam(self.parameters(), lr=learning_rate)

    def test_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])
        new_label = torch.tensor([id_to_label[i.item()] for i in batch["label"]])
        loss = F.cross_entropy(y_hat.cpu(), new_label.cpu())

        # logs metrics for each testing_step,
        # and the average across the epoch, to the progress bar and logger
        self.log("test_loss", loss.item(), batch_size=batch_size)

        accM2 = mcaM2(y_hat.cpu(), new_label.cpu())
        self.log("test_accuracy_micro", accM2.item(), batch_size=batch_size)

        new_label = new_label.type(torch.IntTensor)

        auc = roc_auc(y_hat.cpu(), new_label.cpu())
        self.log("auc", auc.item(), batch_size=batch_size)

        precision = pre(y_hat.cpu(), new_label.cpu())
        recall = rec(y_hat.cpu(), new_label.cpu())
        f1 = f1score(y_hat.cpu(), new_label.cpu())

        self.log("precision", precision.item(), batch_size=batch_size)
        self.log("recall", recall.item(), batch_size=batch_size)
        self.log("f1", f1.item(), batch_size=batch_size)

        print("test_loss:", loss.item(), "test_accuracy_micro:", accM2.item(), "auc:", auc.item(), "precision:", precision.item(), "recall", recall.item(), "f1", f1.item())

        return loss

# Training

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping

early_stopping_callbacks = EarlyStopping(monitor="val_loss", min_delta=0, patience=10, verbose=True, mode="min")

In [ ]:
classification_module = VideoClassificationLightningModule()
data_module = VideosDataModule()
trainer = pytorch_lightning.Trainer(
    default_root_dir=checkpoint_path,
    max_epochs=30,
    accelerator="auto",
    devices="auto",
    strategy='auto',
    enable_checkpointing=True,
    logger=True,
)

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
torch.set_float32_matmul_precision('medium')

In [ ]:
trainer.fit(classification_module, data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type | Params
-------------------------------
0 | model | Net  | 3.0 M 
-------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
11.989    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

val_loss: 4.197101593017578 val_accuracy_micro: 0.8409090638160706
val_loss: 3.5975899696350098 val_accuracy_micro: 0.8636363744735718


Training: |          | 0/? [00:00<?, ?it/s]

train_loss: 5.0946879386901855 train_accuracy_micro: 0.8068181872367859
train_loss: 4.486010551452637 train_accuracy_micro: 0.8295454382896423
train_loss: 4.498588562011719 train_accuracy_micro: 0.8295454382896423
train_loss: 5.384158611297607 train_accuracy_micro: 0.7954545617103577
train_loss: 4.184772968292236 train_accuracy_micro: 0.8409090638160706
train_loss: 5.382187366485596 train_accuracy_micro: 0.7954545617103577
train_loss: 4.768692493438721 train_accuracy_micro: 0.8181818127632141
train_loss: 4.7872161865234375 train_accuracy_micro: 0.8181818127632141
train_loss: 4.7805867195129395 train_accuracy_micro: 0.8181818127632141
train_loss: 5.640738487243652 train_accuracy_micro: 0.7840909361839294
train_loss: 3.2749950885772705 train_accuracy_micro: 0.875
train_loss: 5.960780620574951 train_accuracy_micro: 0.7727272510528564
train_loss: 5.060233116149902 train_accuracy_micro: 0.8068181872367859
train_loss: 6.278525352478027 train_accuracy_micro: 0.7613636255264282
train_loss: 4.1

...

Validation: |          | 0/? [00:00<?, ?it/s]

val_loss: 3.9546127319335938 val_accuracy_micro: 0.8409090638160706
val_loss: 5.570075035095215 val_accuracy_micro: 0.7613636255264282
val_loss: 3.5072269439697266 val_accuracy_micro: 0.8863636255264282
val_loss: 2.5472006797790527 val_accuracy_micro: 0.9204545617103577
val_loss: 4.672828197479248 val_accuracy_micro: 0.7840909361839294
val_loss: 4.558806419372559 val_accuracy_micro: 0.8181818127632141
val_loss: 4.865316867828369 val_accuracy_micro: 0.8181818127632141
val_loss: 4.900625228881836 val_accuracy_micro: 0.7727272510528564
val_loss: 4.98087739944458 val_accuracy_micro: 0.7954545617103577
val_loss: 4.235627174377441 val_accuracy_micro: 0.8522727489471436
val_loss: 3.492455244064331 val_accuracy_micro: 0.875
val_loss: 3.825906276702881 val_accuracy_micro: 0.875
val_loss: 3.5923047065734863 val_accuracy_micro: 0.8522727489471436
val_loss: 4.9620537757873535 val_accuracy_micro: 0.8068181872367859
val_loss: 4.9263916015625 val_accuracy_micro: 0.8181818127632141
val_loss: 4.0453138

Validation: |          | 0/? [00:00<?, ?it/s]

val_loss: 4.220616817474365 val_accuracy_micro: 0.8409090638160706
val_loss: 5.825044631958008 val_accuracy_micro: 0.7272727489471436
val_loss: 3.4006526470184326 val_accuracy_micro: 0.8522727489471436
val_loss: 5.5158610343933105 val_accuracy_micro: 0.7954545617103577
val_loss: 3.793897867202759 val_accuracy_micro: 0.8863636255264282
val_loss: 3.4965922832489014 val_accuracy_micro: 0.8863636255264282
val_loss: 4.087390422821045 val_accuracy_micro: 0.8522727489471436
val_loss: 4.360023021697998 val_accuracy_micro: 0.8295454382896423
val_loss: 4.745967388153076 val_accuracy_micro: 0.8409090638160706
val_loss: 4.383274078369141 val_accuracy_micro: 0.8522727489471436
val_loss: 4.0199785232543945 val_accuracy_micro: 0.8295454382896423
val_loss: 5.7991943359375 val_accuracy_micro: 0.7272727489471436
val_loss: 4.648979663848877 val_accuracy_micro: 0.8068181872367859
val_loss: 2.4460384845733643 val_accuracy_micro: 0.9431818127632141
val_loss: 4.429753303527832 val_accuracy_micro: 0.840909063

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


# Testing


In [ ]:
best_checkpoint_path = '/content/drive/MyDrive/Research/Muscle Video/Checkpoints/Multilabel/x3d_s/not pretrained/lightning_logs/version_0/checkpoints/epoch=29-step=3900.ckpt'

In [ ]:
trainer.test(model=classification_module, datamodule=data_module, ckpt_path=best_checkpoint_path, verbose=True)

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/Research/Muscle Video/Checkpoints/Multilabel/x3d_s/not pretrained/lightning_logs/version_0/checkpoints/epoch=29-step=3900.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/drive/MyDrive/Research/Muscle Video/Checkpoints/Multilabel/x3d_s/not pretrained/lightning_logs/version_0/checkpoints/epoch=29-step=3900.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

test_loss: 3.367461681365967 test_accuracy_micro: 0.8636363744735718 auc: 0.7149122953414917 precision: 0.1515151560306549 recall 0.1818181872367859 f1 0.16363635659217834
test_loss: 4.209471225738525 test_accuracy_micro: 0.8522727489471436 auc: 0.7570776343345642 precision: 0.1515151560306549 recall 0.12121212482452393 f1 0.13333332538604736
test_loss: 5.799005031585693 test_accuracy_micro: 0.8295454382896423 auc: 0.8081023693084717 precision: 0.3181818127632141 recall 0.25757575035095215 f1 0.2666666507720947
test_loss: 3.079418182373047 test_accuracy_micro: 0.9204545617103577 auc: 0.8673245906829834 precision: 0.1818181872367859 recall 0.12878787517547607 f1 0.15064933896064758
test_loss: 3.6032328605651855 test_accuracy_micro: 0.8522727489471436 auc: 0.6953846216201782 precision: 0.20454546809196472 recall 0.19696970283985138 f1 0.16969695687294006
test_loss: 6.294355392456055 test_accuracy_micro: 0.7159090638160706 auc: 0.5415778160095215 precision: 0.022727273404598236 recall 0.0

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│            auc            │    0.7080537676811218     │
│            f1             │    0.14046895503997803    │
│         precision         │    0.16257573664188385    │
│          recall           │    0.14099997282028198    │
│    test_accuracy_micro    │     0.838181734085083     │
│         test_loss         │    4.2925639152526855     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 4.2925639152526855,
  'test_accuracy_micro': 0.838181734085083,
  'auc': 0.7080537676811218,
  'precision': 0.16257573664188385,
  'recall': 0.14099997282028198,
  'f1': 0.14046895503997803}]

In [ ]:
prediction_model = VideoClassificationLightningModule.load_from_checkpoint(best_checkpoint_path)
prediction_model.to(device)
prediction_model.freeze()

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [ ]:
'''
for param in model.backbone.parameters():
    param.requires_grad = False
'''

In [ ]:
testing_dir = os.path.join(input_dir, "test")
#testing_dir = "/content/drive/MyDrive/Research/Muscle Video/Datasets/Muscle activation test videos"
numSeen = 0
numCorrectTop1 = 0
numCorrectTop5 = 0
classWiseTop1 = {}
classWiseTop5 = {}

for i in pred_to_class:
  classWiseTop1[pred_to_class[i]] = 0
  classWiseTop5[pred_to_class[i]] = 0

for folder in os.listdir(testing_dir):
  new_path = os.path.join(testing_dir, folder)
  for file_name in os.listdir(new_path):
    video_path = os.path.join(new_path, file_name)

    start_sec = 0
    end_sec = start_sec + clip_duration

    video = EncodedVideo.from_path(video_path)

    video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

    video_data = transform(video_data)

    inputs = video_data["video"]
    inputs = [i.to(device)[None, ...] for i in inputs]

    preds = prediction_model(inputs)

    preds = post_act(preds)
    pred_classes = preds.topk(k=5).indices

    pred_class_names = [pred_to_class[int(i)] for i in pred_classes[0]]

    numSeen += 1

    if pred_class_names[0] == folder:
        numCorrectTop1 += 1
        classWiseTop1[folder] += 1

    if folder in pred_class_names:
        numCorrectTop5 += 1
        classWiseTop5[folder] += 1

    print("Current Top1:", (numCorrectTop1 / numSeen) * 100, "%")
    print(video_path)
    print(folder)
    print(pred_class_names)

print("Evaluation Finished")
top1accuracy = (numCorrectTop1 / numSeen) * 100
top5accuracy = (numCorrectTop5 / numSeen) * 100
print("Top 1 Accuracy:", top1accuracy, "%")
print("Top 5 Accuracy:", top5accuracy, "%")

Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/split_workout_videos_v1/test/bench press/bench press_57_0.mp4
bench press
['shoulders', 'triceps', 'chest', 'biceps', 'lats']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/split_workout_videos_v1/test/bench press/bench press_57_1.mp4
bench press
['shoulders', 'triceps', 'chest', 'lats', 'biceps']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/split_workout_videos_v1/test/bench press/bench press_18_0.mp4
bench press
['triceps', 'shoulders', 'chest', 'biceps', 'lats']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/split_workout_videos_v1/test/bench press/bench press_30_0.mp4
bench press
['triceps', 'chest', 'shoulders', 'lats', 'biceps']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/split_workout_videos_v1/test/bench press/bench press_26_0.mp4
bench press
['triceps', 'chest', 'shoulders', 'biceps', 'lats']


In [ ]:
testing_dir = "/content/drive/MyDrive/Research/Muscle Video/Datasets/Muscle activation test videos"
numSeen = 0
numCorrectTop1 = 0
numCorrectTop5 = 0
classWiseTop1 = {}
classWiseTop5 = {}

for i in pred_to_class:
  classWiseTop1[pred_to_class[i]] = 0
  classWiseTop5[pred_to_class[i]] = 0

for folder in os.listdir(testing_dir):
  new_path = os.path.join(testing_dir, folder)
  for file_name in os.listdir(new_path):
    video_path = os.path.join(new_path, file_name)

    start_sec = 0
    end_sec = start_sec + clip_duration

    video = EncodedVideo.from_path(video_path)

    video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

    video_data = transform(video_data)

    inputs = video_data["video"]
    inputs = [i.to(device)[None, ...] for i in inputs]

    preds = prediction_model(inputs)

    preds = post_act(preds)
    pred_classes = preds.topk(k=5).indices

    pred_class_names = [pred_to_class[int(i)] for i in pred_classes[0]]

    numSeen += 1

    if pred_class_names[0] == folder:
        numCorrectTop1 += 1
        classWiseTop1[folder] += 1

    if folder in pred_class_names:
        numCorrectTop5 += 1
        classWiseTop5[folder] += 1

    print("Current Top1:", (numCorrectTop1 / numSeen) * 100, "%")
    print(video_path)
    print(folder)
    print(pred_class_names)

print("Evaluation Finished")
top1accuracy = (numCorrectTop1 / numSeen) * 100
top5accuracy = (numCorrectTop5 / numSeen) * 100
print("Top 1 Accuracy:", top1accuracy, "%")
print("Top 5 Accuracy:", top5accuracy, "%")

Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/Muscle activation test videos/Personal Test 1/20231105_150207.mp4
Personal Test 1
['shoulders', 'lats', 'biceps', 'abs', 'chest']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/Muscle activation test videos/Personal Test 1/spin thing obliques.mp4
Personal Test 1
['abs', 'obliques', 'triceps', 'chest', 'shoulders']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/Muscle activation test videos/Personal Test 1/reverse curl.mp4
Personal Test 1
['biceps', 'abs', 'obliques', 'lats', 'triceps']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/Muscle activation test videos/Personal Test 1/hammer curl.mp4
Personal Test 1
['biceps', 'lats', 'abs', 'obliques', 'triceps']
Current Top1: 0.0 %
/content/drive/MyDrive/Research/Muscle Video/Datasets/Muscle activation test videos/Personal Test 1/lunges.mp4
Personal Test 1
['quads', 'hamstrings', 'glutes',